In [ ]:
%run ../../lib/basic-import.ipynb
%run ../../lib/plot-functions.ipynb

In [ ]:
import termtables as tt

#pip install termtables

In [ ]:
def pd_plot_lap_save_corr():
    add_df = pd.concat(all_dfs, axis=1, keys=[a[:-4] for a in participant_files])
    ID = '50338'
    rightPosition = 1.00
    leftPosition = -0.10

    fig, ax = plt.subplots()
    fig.subplots_adjust(right=2)

    ax2 = ax.twinx()
    ax3 = ax.twinx()
    ax4 = ax.twinx()
    ax5 = ax.twinx()
    ax6 = ax.twinx()

    rightPosition = ax_modification(ax2,'right', rightPosition)
    rightPosition = ax_modification(ax3,'right', rightPosition)
    leftPosition = ax_modification(ax4,'left', leftPosition)
    rightPosition = ax_modification(ax5,'right', rightPosition)
    leftPosition = ax_modification(ax6,'left', leftPosition)

    lapDates = add_df[ID].index[add_df[ID]['lap'].diff()==1]
    #create new pd lap_df. Contains only +30, -15 sec laps, 1 ekstra for nans.
    #lap_df, gir en ny dataframe. Denne inneholder de tre lap_dfs dataframe.
    lap_dfs = get_lapdf(add_df, ID, 15, 32, lapDates)
    lap_df = pd.concat(lap_dfs)
    lap_df.index = map(lambda x: str(x)[-8:], lap_df.index)

    lap_df['accumulated_power'].plot(ax=ax, color='red',legend=False, title="all laps -15,+30 pd 50338")
    lap_df['speed'].plot(ax=ax2, color='blue',legend=False)
    lap_df['power'].plot(ax=ax3, color='green',legend=False)
    lap_df['altitude'].plot(ax=ax4, color='orange',legend=False)
    lap_df['distance'].plot(ax=ax5, color='lightcoral',legend=False)
    lap_df['heart_rate'].plot(ax=ax6, color='purple',legend=False)

    ax_style(ax,'Accumulated Power','red')
    ax_style(ax2,'Speed','blue')
    ax_style(ax3,'Power','green')
    ax_style(ax4,'Altitude','orange')
    ax_style(ax5,'Distance','lightcoral')
    ax_style(ax6,'Heartrate','purple')

    #getting correlation values, now values and changeable values.
    nowVals = get_now_val(lapDates, ID, add_df)
    corrVals = get_corr_val(lap_dfs)
    changVals = [date.date() for date in lapDates]

    #allVals is the values over in one array.
    allVals = []
    for i in range(len(corrVals)):
        allVals.append([changVals[i]] + [corrVals[i]] + nowVals[i])

    #table which is shown. might change the names.
    headerTable = ['timestamp correlation changeable','timestamp correlation suggestion','timestamp now','lap','distance']
    print(tt.to_string(allVals, header=headerTable))

    #tableDF, used in csv.
    dataTableDF = [headerTable] + allVals
    tableDF = pd.DataFrame(dataTableDF[1:],columns=dataTableDF[0]).set_index('timestamp correlation changeable')
    tableDF.to_csv('correlation_{}'.format(ID))


pd_plot_lap_save_corr()

In [ ]:
def pd_plot_corr():
    add_df = pd.concat(all_dfs, axis=1, keys=[a[:-4] for a in participant_files])
    ID = '50338'
    rightPosition = 1.00
    leftPosition = -0.10

    fig, ax = plt.subplots()
    fig.subplots_adjust(right=2)

    ax2 = ax.twinx()
    ax3 = ax.twinx()
    ax4 = ax.twinx()
    ax5 = ax.twinx()
    ax6 = ax.twinx()

    rightPosition = ax_modification(ax2,'right', rightPosition)
    rightPosition = ax_modification(ax3,'right', rightPosition)
    leftPosition = ax_modification(ax4,'left', leftPosition)
    rightPosition = ax_modification(ax5,'right', rightPosition)
    leftPosition = ax_modification(ax6,'left', leftPosition)

    lapDates = add_df[ID].index[add_df[ID]['lap'].diff()==1]

    corrDF = pd.read_csv('correlation_{}'.format(ID)).set_index("timestamp correlation changeable")
    corrTS = get_corrTS(corrDF)

    lap_dfs = get_lapdf(add_df, ID, 15, 32, corrTS)
    lap_df = pd.concat(lap_dfs)
    lap_df.index = map(lambda x: str(x)[-8:], lap_df.index)


    lap_df['accumulated_power'].plot(ax=ax, color='red',legend=False, title="corr laps -15,+30 pd 50338")
    lap_df['speed'].plot(ax=ax2, color='blue',legend=False)
    lap_df['power'].plot(ax=ax3, color='green',legend=False)
    lap_df['altitude'].plot(ax=ax4, color='orange',legend=False)
    lap_df['distance'].plot(ax=ax5, color='lightcoral',legend=False)
    lap_df['heart_rate'].plot(ax=ax6, color='purple',legend=False)

    ax_style(ax,'Accumulated Power','red')
    ax_style(ax2,'Speed','blue')
    ax_style(ax3,'Power','green')
    ax_style(ax4,'Altitude','orange')
    ax_style(ax5,'Distance','lightcoral')
    ax_style(ax6,'Heartrate','purple')

pd_plot_corr()